Chinese character recognition competition on Kaggle --> https://www.kaggle.com/c/chinese-char-recognition-smmo19/leaderboard

In [ ]:
! mkdir ~/.kaggle/ && echo '{"username":"...","key":"..."}' > ~/.kaggle/kaggle.json

In [ ]:
import kaggle
kaggle.api.competition_download_files('chinese-char-recognition-smmo19') 
! kaggle competitions download -c chinese-char-recognition-smmo19

In [ ]:
! unzip test.npy.zip

In [ ]:
! unzip train-1.npy.zip

In [ ]:
! unzip train-2.npy.zip

In [ ]:
! unzip train-3.npy.zip

In [ ]:
! unzip train-4.npy.zip

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
data = np.load("train-1.npy", allow_pickle=True)

for i in range(1, 5):
  data = np.concatenate([data, np.load(f'train-{i}.npy', allow_pickle=True)])

data_test = np.load("test.npy", allow_pickle=True)
np.random.shuffle(data) 

In [ ]:
batch_size = 256
val_size = 0.2
INIT_LR = 5e-4
EPOCHS = 20

In [ ]:
st_per_ep = int((1-val_size)*data.shape[0]/(batch_size))
st_per_ep

In [ ]:
val_st = int(val_size*data.shape[0]/(batch_size))
val_st

In [ ]:
char_to_id = {char:id for id, char in enumerate(np.unique([char for _, char in data]))}
id_to_char = {id:char for id, char in enumerate(np.unique([char for _, char in data]))}

In [ ]:
def train_gen():
  for img, label in data[int((val_size * len(data))):]:
    img = img[:, :, np.newaxis]
    label = char_to_id[label]
    yield img, label

def val_gen():
  for img, label in data[:int((val_size * len(data)))]:
    img = img[:, :, np.newaxis]
    label = char_to_id[label]
    yield img, label

def test_gen():
  for img in data_test:
    img = img[:, :, np.newaxis]
    yield img

In [ ]:
dataset_train = tf.data.Dataset.from_generator(
    train_gen,
    output_types=(tf.uint8, tf.int32),
    output_shapes=((None, None, 1), ())
)

dataset_val = tf.data.Dataset.from_generator(
    val_gen,
    output_types=(tf.uint8, tf.int32),
    output_shapes=((None, None, 1), ())
)

dataset_test = tf.data.Dataset.from_generator(
    test_gen,
    output_types=tf.uint8,
    output_shapes=(None, None, 1)
)

In [ ]:
# def augmentation(img):
#   rand = np.random.rand()
#   if rand <= 0.25:
#     tf.image.random_brightness(img, 0.3)
#     return img
#   if rand <= 0.5: 
#     tf.image.adjust_gamma(img, 0.3)
#     return img
#   if rand <= 0.75:
#     tf.image.apply_affine_transform(img, shear=35)
#     return img
#   if rand <= 1:
#     tf.image.apply_brightness_shift(img, 0.5)
#     return img

def preprocess_train(img, label):
  img = tf.image.resize(img, (80, 80))
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  if np.random.rand() < 0.25:
    img = augmentation(img)
  label = tf.one_hot(label, 1000)
  return img, label

def preprocess_test(img):
  img = tf.image.resize(img, (80, 80))
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  return img



In [ ]:
dataset_train = dataset_train.map(preprocess_train, num_parallel_calls=-1).shuffle(1024).repeat(EPOCHS).prefetch(-1).batch(batch_size)

In [ ]:
dataset_val = dataset_val.map(preprocess_train, num_parallel_calls=-1).shuffle(1024).repeat(EPOCHS).prefetch(-1).batch(batch_size)
dataset_test = dataset_test.map(preprocess_test, num_parallel_calls=-1).prefetch(-1).batch(batch_size)

In [ ]:
initializer = tf.keras.initializers.lecun_normal()

In [ ]:
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc',
#                            patience=2,
#                            verbose=1,
#                            min_delta=1e-3)
def lr_scheduler(epoch):
    return INIT_LR * 0.8 ** epoch 
import keras
from keras.models import save_model, load_model

class ModelSaveCallback(keras.callbacks.Callback):

    def __init__(self, file_name):
        super(ModelSaveCallback, self).__init__()
        self.file_name = file_name

    def on_epoch_end(self, epoch, logs=None):
        self.model.save(self.file_name.format(epoch), overwrite=True)
        print("model was saved to file ", self.file_name.format(epoch))

def load_from_file(model_filename, last_epoch):
  return(load_model(model_filename.format(last_epoch-1))) ##кажется, было слишком легко, надеюсь, все правильно сделала

In [ ]:
# ready_arch_model = tf.keras.applications.resnet_v2.ResNet101V2(input_shape=(80, 80, 1),
#                                                include_top=True,
#                                                weights=None) - не пробил бейзлайн на 10

ready_arch_model = tf.keras.applications.resnet_v2.ResNet101V2(input_shape=(80, 80, 1),
                                               include_top=True,
                                               weights=None)- тоже не пробил, но последний сабмит был с ним

# ready_arch_model = tf.keras.applications.xception.Xception(input_shape=(80, 80, 1),
#                                                include_top=True,
#                                                weights=None) - вот этот пробовала запускать, 
#но не доучился из-за ошибки runtime, последняя запущенная сессия этого ноутбука как раз с ним


In [ ]:
ready_arch_model.trainable = True

model = tf.keras.Sequential([
  ready_arch_model,
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())


In [ ]:
model.fit(dataset_train, validation_data = dataset_val, epochs=EPOCHS, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler), ModelSaveCallback('weights_{0:02d}.hdf5')], steps_per_epoch=st_per_ep, validation_steps=val_st)

In [ ]:
res = pd.read_csv('random_labels.csv')

In [ ]:
full_prediction = model.predict(dataset_test)

In [ ]:
prediction = full_prediction.argmax(1)

In [ ]:
res.shape, prediction.shape

In [ ]:
answ = []
for i in prediction:
  answ.append(id_to_char[i])

In [ ]:
res['Category'] = answ
res

In [ ]:
res.to_csv("result.csv", index=False)

In [ ]:
! kaggle competitions submit -c chinese-char-recognition-smmo19 -m '' -f result.csv